In [1]:
from googletrans import Translator
import openpyxl
import time
from langdetect import detect, LangDetectException

In [2]:
def translate(workbook, saveStr: str, worksheet, starting_row: int, lang: str):
    # Initialize the translator
    translator = Translator()

    for row in range(starting_row, worksheet.max_row + 1):
        try:
            if row % 250 == 0:
                print(row)
            if row % 5000 == 0:
                print('Buffering...')
                workbook.save(saveStr)

            # Get the text of the cell
            cell_value = worksheet.cell(row=row, column=3).value

            if lang != '':
                translated_text = translator.translate(cell_value, src=lang, dest='en').text
            else:
                # Get the language of the text
                src_lang = detect(cell_value)
                if src_lang != 'en':
                    translated_text = translator.translate(cell_value, src=src_lang, dest='en').text
                else:
                    continue

            # Write the translated text to the corresponding cell in the column to write
            worksheet.cell(row=row, column=10).value = translated_text

        except (IndexError, TypeError, LangDetectException):
            continue
        except Exception as e:
            print(e)
            print('The exception occurred in row ' + str(row) + '. Trying again.')
            time.sleep(5)
            if not translate(workbook, saveStr, worksheet, row, lang):
                return False
            else:
                break
        except KeyboardInterrupt:
            print('Interrupted in row ' + str(row) + '.')
            return False

    return True

In [ ]:
# Code base

workbook = openpyxl.load_workbook('SOURCE PATH')

for name in workbook.sheetnames:
    # Select the sheet you want to read and write to
    worksheet = workbook[name]

    # Select where to save the workbook
    saveStr = 'SAVE PATH'

    # Give the column for the translations a title
    worksheet.cell(row=1, column=10).value = 'translation'

    print('Translating...')

    # Iterate over each cell in the column to read and write the translated value
    if not translate(workbook, saveStr, worksheet, 2, ''):
        print('Something went wrong.')
    else:
        print('Translation done!')

# Save the changes to the workbook and close it afterwards
workbook.save(saveStr)
workbook.close()
print('The workbook has been saved.')